Process curriculum and coding data ready to be fed into the model


In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict

In [2]:
institutions = ["CAU_Germany",
                "Durham_England",
                "Edinburgh_Scotland",
                "Glasgow_Scotland",
                "Kennesaw_USA",
                "UOC_Spain",
                "UofT_Canada",
                "Uppsala_Sweden",
                "VirginiaTech_USA"
               ]

In [3]:
all_electives = pd.read_csv(os.path.join("curriculum_content", "all_electives.csv"), dtype = 'string')

In [4]:
inst_enrolment_dtypes = {"MCode": 'string', "Cohort": 'string', "AcademicYearStart":'int', "Women3": 'int', "Men3": 'int', "Cap": 'object', "institution": 'string'}
inst_enrolment_columns = list(inst_enrolment_dtypes.keys())
cohort_enrolment_dtypes = {"Cohort": 'string', "AcademicYearStart": 'int', "Women3": 'int', "Men3": 'int', "MaxElectives": 'object', "institution": 'string'}
cohort_enrolment_columns = list(cohort_enrolment_dtypes.keys())
inst_enrolments = []
cohort_enrolments = []
for institution in institutions:
    print ("loading " + institution)
    try:
        inst_enrolment = pd.read_csv(os.path.join("enrolment", institution, "elective_enrolment_rounded.csv"), dtype = inst_enrolment_dtypes)
        inst_enrolment['institution'] = institution
        inst_enrolments.append(inst_enrolment)
    except FileNotFoundError:
        print ("enrolment file Not Found for " + institution)
    try:
        cohort_enrolment = pd.read_csv(os.path.join("enrolment", institution, "cohort_enrolment_rounded.csv"), dtype = cohort_enrolment_dtypes)
        cohort_enrolment['institution'] = institution
        cohort_enrolments.append(cohort_enrolment)
    except FileNotFoundError:
        print ("cohort enrolment file Not Found for " + institution)        

all_enrolments = pd.concat(inst_enrolments)
all_enrolments['Students3'] = all_enrolments['Women3'] + all_enrolments['Men3']
all_enrolments.to_csv(os.path.join("enrolment", "all_elective_enrolment_rounded.csv"), index = False)

all_cohort_enrolments = pd.concat(cohort_enrolments)
all_cohort_enrolments.to_csv(os.path.join("enrolment", "all_cohort_enrolment_rounded.csv"), index = False)

loading CAU_Germany
loading Durham_England
loading Edinburgh_Scotland
loading Glasgow_Scotland
loading Kennesaw_USA
loading UOC_Spain
loading UofT_Canada
loading Uppsala_Sweden
loading VirginiaTech_USA


In [5]:
all_electives_codes = all_electives['elective']
all_enrolments_codes = all_enrolments['MCode'].unique()
electives_without_enrolment = list(set(all_electives_codes) - set(all_enrolments_codes))
print(len(electives_without_enrolment))
print(len(all_enrolments_codes))
electives_without_enrolment


14
273


['cxinfr11158',
 'cxinfr11129',
 'COMP3637',
 '1DL321',
 'cxinfr08020',
 '1DL301',
 'COMPSCI4094',
 'cxinfr08010',
 'CS4884',
 'cxinfr10084',
 '1DL311',
 '1DL231',
 'CS4824',
 'cxinfr08033']

In [6]:
def make_code_columns(raw_coded):
    meta_cols = []
    coder_cols = []
    renamer = {}
    for col in raw_coded.columns:
        shortened = col
        if col[0:3] == "[M]":
            shortened = col[4:-2]
            meta_cols.append(shortened)
        if col[0:3] == "[C]":
            shortened = col[3:]
            coder_cols.append(shortened)
        renamer[col] = shortened


    # first two meta columns are elective code and institution, which we use as the index
    # third meta column is title
    index_cols = meta_cols[0:3]
    keep_cols = index_cols + coder_cols
    base = raw_coded.rename(columns=renamer)[keep_cols]

    long1 = base.melt(index_cols)
    long1.value = long1.value.str.replace(" ","")
    long1.value = long1.value.str.pad(width=2, side='left', fillchar='0')
    long1.value = long1.value.str.replace("00","")

    long2 = long1.join(long1['value'].str.split(';', expand=True))
    long3 = long2.drop('value', axis=1)
    long3 = long3.rename(columns={'variable': 'coder'})

    long4 = long3.melt(index_cols + ['coder'])
    long4 = long4.rename(columns={'value': 'code'})[index_cols + ['coder','code']]
    long4.fillna("",inplace = True)
    long4 = long4[long4.code.str.len()>0]

    long5 = long4.groupby(index_cols + ['code']).count().unstack().fillna(0)
    long5.columns = long5.columns.get_level_values(1)
    return long5


In [7]:
def has_agreement_inset_code(code_in_columns, value_set, code):
    return code_in_columns[code].isin(value_set)

def has_agreement_inset_any_code(code_in_columns, value_set):
    has_agreement = False
    for col in code_in_columns.columns:
        has_agreement = has_agreement | has_agreement_inset_code(code_in_columns, value_set, col)
    return has_agreement

def has_agreement_4_plus_code(code_in_columns, code):
    return has_agreement_inset_code(code_in_columns, [4,5,6,7], code)

def has_agreement_23_code(code_in_columns, code):
    return has_agreement_inset_code(code_in_columns, [2,3], code)

def has_agreement_3_code(code_in_columns, code):
    return has_agreement_inset_code(code_in_columns, [3], code)

def has_agreement_4_plus_any_code(code_in_columns):
    return has_agreement_inset_any_code(code_in_columns, [4,5,6,7])

def has_agreement_23_any_code(code_in_columns):
    return has_agreement_inset_any_code(code_in_columns, [2,3])

def has_agreement_3_any_code(code_in_columns):
    return has_agreement_inset_any_code(code_in_columns, [3])

def has_good_overall_agreement_code(code_in_columns, code):
    return (has_agreement_4_plus_code(code_in_columns, code)) & ~(has_agreement_23_code(code_in_columns, code))

def has_good_overall_agreement_any_code(code_in_columns):
    return (has_agreement_4_plus_any_code(code_in_columns)) & ~(has_agreement_23_any_code(code_in_columns))

def has_some_overall_agreement_code(code_in_columns, code):
    return (has_agreement_4_plus_code(code_in_columns, code)) & ~(has_agreement_3_code(code_in_columns, code))

def has_some_overall_agreement_any_code(code_in_columns):
    return (has_agreement_4_plus_any_code(code_in_columns)) & ~(has_agreement_3_any_code(code_in_columns))


In [8]:
stage_one_files = ["20230622WG6_AllElectivesPart1-stage1.csv", "20230622WG6_AllElectivesPart2-stage1.csv", "20230622WG6_AllElectivesPart3-stage1.csv", "20230825WG6_AllElectivesPart4-stage1.csv", "20230907WG6_AllElectivesPart5-stage1.csv"]

stage_one_contents = []
for stage_one_file in stage_one_files:
    raw = pd.read_csv(os.path.join("curriculum_content","coded", "stage1", stage_one_file), dtype='string').fillna("")
    raw = raw.replace("Edinburgh_UK", "Edinburgh_Scotland")
    raw = raw.replace("\n"," ")
    stage_one_contents.append(raw)
raw_coded = pd.concat(stage_one_contents)
raw_coded.to_csv(os.path.join("curriculum_content","coded","stage1_codes_raw.csv"))

code_in_columns = make_code_columns(raw_coded)
code_in_columns.to_csv(os.path.join("curriculum_content","coded","stage1_codes_columns.csv"))


In [9]:
all_stage1_coded_codes = code_in_columns.reset_index()['elective']
print(len(all_stage1_coded_codes))
set(all_enrolments_codes) - set (all_stage1_coded_codes)

294


set()

In [10]:
all_enrolments_codes

<StringArray>
[  'Inf-NumProg',      'Inf-NNDL',    'inf-EnlOpt',      'inf-EAeS',
 'Inf-GraphDraw',        'inf-TM',   'inf-AuLearn',        'inf-CI',
    'Inf-DigSig',       'inf-IoT',
 ...
        'CS4254',        'CS4264',        'CS4284',        'CS4504',
        'CS4604',        'CS4624',        'CS4644',        'CS4704',
        'CS4784',        'CS4804']
Length: 273, dtype: string

In [11]:
code_in_columns.columns

Index(['01', '02', '03', '04', '07', '08', '09', '10', '13', '14', '15', '16',
       '17', '18', '19', '21', '22', '24', 'AI', 'AL', 'AR', 'DM', 'FPL',
       'GIT', 'HCI', 'MSF', 'NC', 'OS', 'PDC', 'SDF', 'SE', 'SEC', 'SEP', 'SF',
       'SPD'],
      dtype='object', name='code')

In [12]:
set (all_stage1_coded_codes) - set(all_enrolments_codes)

{'1DL231',
 '1DL301',
 '1DL311',
 '1DL321',
 'COMP3637',
 'COMPSCI4094',
 'CS4824',
 'CS4884',
 'cxinfr08010',
 'cxinfr08020',
 'cxinfr08033',
 'cxinfr10075',
 'cxinfr10083',
 'cxinfr10084',
 'cxinfr11077',
 'cxinfr11124',
 'cxinfr11129',
 'cxinfr11158',
 'cxinfr11215',
 'cxinfr11240',
 'cxinfr11241'}

In [13]:
# update the elective coding with a specific subset (rows: institution and elective, columns: topics/application areas)
def add_coding (original, update):
    # from https://stackoverflow.com/questions/38543263/pandas-multiply-all-the-numeric-values-in-the-data-frame-by-a-constant
    update[update.select_dtypes(include=['number']).columns] *= 7
    key_columns = ['institution', 'elective']
    updated_columns = list(set(update.columns) - set(key_columns))
#    print (updated_columns, "updated columns for ", stage_two_file)
    joined = original.merge(update, on=key_columns, how='left')
    for updated_column in updated_columns:
        try:
            joined[updated_column+"_y"] = joined[updated_column+"_y"].fillna(joined[updated_column+"_x"])
        except BaseException:
            print ("Exception")
            print ("joined _y",joined[updated_column+"_y"])
            print ("joined _x",joined[updated_column+"_x"])
            break
        joined[updated_column] = joined[updated_column+"_y"].astype('float')
        joined.drop(columns=[updated_column + "_x", updated_column + "_y"], inplace=True)
    return joined.fillna(0)

In [14]:
types = defaultdict(float, institution='str', elective='str')
test = pd.read_csv(os.path.join("curriculum_content", "coded","stage1_codes_columns.csv"), dtype = types, keep_default_na=False)
test

test = test[test['institution'] == "CAU_Germany"]
print(test)
test_no_match = pd.DataFrame({"elective":["1DL034"], "institution":["Upps4l4_Sweden"], "PDC":[1]})
test_update = pd.DataFrame({"elective":["1DL034"], "institution":["Uppsala_Sweden"], "PDC":[1]})
test_no_majority = pd.read_csv(os.path.join("curriculum_content", "coded", "stage2", "stage2_codes_no_majority.csv"), dtype = types, keep_default_na=False)

test_result = add_coding(test, test_no_majority)
test_result[["elective", "17"]]

          elective  institution  \
202  Inf-CombWords  CAU_Germany   
203     Inf-DigSig  CAU_Germany   
204  Inf-GraphDraw  CAU_Germany   
205       Inf-NNDL  CAU_Germany   
206    Inf-NumProg  CAU_Germany   
212      WInf-eBeM  CAU_Germany   
287    inf-AuLearn  CAU_Germany   
288         inf-CI  CAU_Germany   
289       inf-EAeS  CAU_Germany   
290       inf-EERP  CAU_Germany   
291     inf-EnlOpt  CAU_Germany   
292        inf-IoT  CAU_Germany   
293         inf-TM  CAU_Germany   

                                                 title   01   02   03   04  \
202                             Combinatorics on Words  0.0  0.0  0.0  0.0   
203  Multimedia Signal Processing and Data Compression  0.0  0.0  1.0  0.0   
204                            Automatic Graph Drawing  0.0  0.0  0.0  0.0   
205                  Neural networks and deep learning  0.0  0.0  0.0  0.0   
206                              Numerical Programming  0.0  0.0  0.0  0.0   
212                         E-Business un

,elective,17
0,Inf-CombWords,0.0
1,Inf-DigSig,0.0
2,Inf-GraphDraw,0.0
3,Inf-NNDL,0.0
4,Inf-NumProg,0.0
5,WInf-eBeM,7.0
6,inf-AuLearn,0.0
7,inf-CI,0.0
8,inf-EAeS,0.0
9,inf-EERP,4.0


In [15]:
types = defaultdict(float, institution='str', elective='str')

stage_two = pd.read_csv(os.path.join("curriculum_content", "coded","stage1_codes_columns.csv"), dtype = types, keep_default_na=False)


print(len(stage_two.columns))
stage_two_files = ["stage2_codes_columns-DM.csv",
                   "stage2_codes_columns-SDF.csv",
                   "stage2_codes_columns-AL.csv",
                   "stage2_codes_columns-SE-SP.csv",
                   "stage2_codes_columns-SPD.csv",
                   "stage2_codes_columns-FPL.csv",
                   "stage2_codes_columns-MSF.csv",
                   "stage2_codes_columns-GIT.csv",
                   "stage2_codes_columns-CAH.csv",
                   "stage2_codes_columns-OS.csv",
                   "stage2_codes_columns-AI.csv",
                   "stage2_codes_columns-AR.csv",
                   "stage2_codes_columns-SEC.csv",
                   "stage2_codes_no_majority.csv",
                   "stage2_codes_no_majority-Part4.csv",
                    "stage2_codes_no_majority-Part5.csv",
                  ]
for stage_two_file in stage_two_files:
    stage_two_addition = pd.read_csv(os.path.join("curriculum_content", "coded", "stage2", stage_two_file), dtype = types, keep_default_na=False)
    stage_two_addition = stage_two_addition.replace("Edinburgh_UK", "Edinburgh_Scotland")
    stage_two = add_coding(stage_two, stage_two_addition)
    
stage_two.to_csv(os.path.join("curriculum_content", "coded","stage2_codes_columns.csv"), index = False)

38


In [16]:
stage_two

,elective,institution,title,07,SE,HCI,15,01,08,DM,...,AR,19,17,SEP,13,AI,24,09,22,OS
0,1DL034,Uppsala_Sweden,Introduction to Machine Learning,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0
1,1DL231,Uppsala_Sweden,Algorithms and Data Structures II,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1DL251,Uppsala_Sweden,Software Engineering and Project Management,0.0,6.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1DL301,Uppsala_Sweden,Database Design I,0.0,0.0,0.0,0.0,0.0,0.0,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1DL311,Uppsala_Sweden,Semantics of Programming Languages,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289,inf-EAeS,CAU_Germany,Methods for Efficient Similarity Search in Lar...,0.0,0.0,0.0,0.0,0.0,0.0,7.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
290,inf-EERP,CAU_Germany,Implementation of ERP systems,0.0,7.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
291,inf-EnlOpt,CAU_Germany,Introduction to Nonlinear Optimization,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
292,inf-IoT,CAU_Germany,Internet of Things and Wireless Networks,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
all_stage2_coded_codes = stage_two['elective']
set(all_enrolments_codes) - set (all_stage2_coded_codes)

set()

In [18]:
elective_enrolment = pd.read_csv(os.path.join("enrolment", "all_elective_enrolment_rounded.csv"))
elective_enrolment[['institution','MCode','Women3','Men3','Students3']].groupby('institution').agg({'MCode': 'count', 'Women3': 'sum', 'Men3': 'sum', 'Students3': 'sum'})

,MCode,Women3,Men3,Students3
institution,,,,
CAU_Germany,13,81,468,549
Durham_England,78,708,3186,3894
Edinburgh_Scotland,102,1395,4371,5766
Glasgow_Scotland,74,1308,4896,6204
Kennesaw_USA,159,1857,6957,8814
UOC_Spain,85,1527,12264,13791
UofT_Canada,34,414,2631,3045
Uppsala_Sweden,45,609,1974,2583
VirginiaTech_USA,66,1209,5217,6426


In [19]:
code_set = pd.read_csv(os.path.join("curriculum_content", "ACM_2023_CAH_codes.csv"), dtype="string")
code_set = code_set.rename(columns={'Abbrev': 'code'})
acm_code_set = code_set[~(code_set.Area == "Non-CS application")]
acm_code_list = list(acm_code_set['code'])
cah_code_set = code_set[code_set.Area == "Non-CS application"]
cah_code_list = list(cah_code_set['code'])
print(acm_code_set.to_latex(index=False))
print(cah_code_set.to_latex(index=False))

\begin{tabular}{lll}
\toprule
                                   Title & code &         Area \\
\midrule
               Algorithms and Complexity &   AL &     Software \\
           Architecture and Organization &   AR &      Systems \\
                 Artificial Intelligence &   AI & Applications \\
                         Data Management &   DM &      Systems \\
    Foundations of Programming Languages &  FPL &     Software \\
     Graphics and Interactive Techniques &  GIT & Applications \\
              Human-Computer Interaction &  HCI & Applications \\
Mathematical and Statistical Foundations &  MSF & Underpinning \\
            Networking and Communication &   NC &      Systems \\
                       Operating Systems &   OS &      Systems \\
      Parallel and Distributed Computing &  PDC &      Systems \\
                                Security &  SEC &      Systems \\
     Society, Ethics and Professionalism &  SEP & Underpinning \\
       Software Development Fundament

/var/folders/my/th4q3hxs0b7c1t1qlv0j2dtm0000gp/T/ipykernel_19901/3152829440.py:7: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(acm_code_set.to_latex(index=False))
/var/folders/my/th4q3hxs0b7c1t1qlv0j2dtm0000gp/T/ipykernel_19901/3152829440.py:8: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(cah_code_set.to_latex(index=False))


In [20]:
dataset_codes = pd.read_csv(os.path.join("curriculum_content","coded","stage2_codes_columns.csv"))
dataset_codes

,elective,institution,title,07,SE,HCI,15,01,08,DM,...,AR,19,17,SEP,13,AI,24,09,22,OS
0,1DL034,Uppsala_Sweden,Introduction to Machine Learning,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0
1,1DL231,Uppsala_Sweden,Algorithms and Data Structures II,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1DL251,Uppsala_Sweden,Software Engineering and Project Management,0.0,6.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1DL301,Uppsala_Sweden,Database Design I,0.0,0.0,0.0,0.0,0.0,0.0,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1DL311,Uppsala_Sweden,Semantics of Programming Languages,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289,inf-EAeS,CAU_Germany,Methods for Efficient Similarity Search in Lar...,0.0,0.0,0.0,0.0,0.0,0.0,7.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
290,inf-EERP,CAU_Germany,Implementation of ERP systems,0.0,7.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
291,inf-EnlOpt,CAU_Germany,Introduction to Nonlinear Optimization,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
292,inf-IoT,CAU_Germany,Internet of Things and Wireless Networks,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
dataset_codes_binary = dataset_codes[['institution','elective']]
for column in list(code_set['code']):
    if column in dataset_codes.columns:
        dataset_codes_binary[column] = dataset_codes[column] >=4
    else:
        print ("No topic codes for " + column)
    
dataset_codes_binary.replace({True: 1, False: 0}, inplace=True)

dataset_codes_binary.to_csv(os.path.join("curriculum_content", "coded", "stage2_coded_binary.csv"))

No topic codes for 05
No topic codes for 06
No topic codes for 12
No topic codes for 20


/var/folders/my/th4q3hxs0b7c1t1qlv0j2dtm0000gp/T/ipykernel_19901/1277088463.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_codes_binary[column] = dataset_codes[column] >=4
/var/folders/my/th4q3hxs0b7c1t1qlv0j2dtm0000gp/T/ipykernel_19901/1277088463.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_codes_binary[column] = dataset_codes[column] >=4
/var/folders/my/th4q3hxs0b7c1t1qlv0j2dtm0000gp/T/ipykernel_19901/1277088463.py:4: SettingWithCopyWarning: 
A value is trying to be set on a co

In [22]:
stage_two[~has_agreement_4_plus_any_code(stage_two)]

,elective,institution,title,07,SE,HCI,15,01,08,DM,...,AR,19,17,SEP,13,AI,24,09,22,OS


In [30]:
all_electives_enrolment = pd.read_csv(os.path.join("enrolment", "all_elective_enrolment_rounded.csv"))
stage_two_enrolment = stage_two.merge(all_electives_enrolment, left_on=['institution','elective'], right_on=['institution','MCode'], how='left')
stage_two_enrolment.sort_values('Students3', ascending=False)
stage_two_enrolment.to_csv(os.path.join("enrolment", "stage_two_enrolment.csv"))

In [31]:
all_cohorts_enrolment = pd.read_csv(os.path.join("enrolment","all_cohort_enrolment_rounded.csv"))
all_cohorts_enrolment.sort_values('Men3', ascending=False)
all_cohorts_enrolment['Men3'].sum() + all_cohorts_enrolment['Women3'].sum()

23096

In [32]:
stage_two_enrolment_cohort = stage_two_enrolment.merge(all_cohorts_enrolment, on=['Cohort','institution','AcademicYearStart'], how='left')


In [33]:
stage_two_enrolment_collapsed = stage_two_enrolment_cohort.groupby('elective')[['Women3_x', 'Men3_x', 'Women3_y', 'Men3_y']].sum()
stage_two_enrolment_collapsed.to_csv("stage_two_enrolment_collapsed.csv")
stage_two_enrolment_collapsed

,Women3_x,Men3_x,Women3_y,Men3_y
elective,,,,
1DL034,57.0,219.0,129.0,394.0
1DL231,0.0,0.0,0.0,0.0
1DL251,129.0,327.0,258.0,788.0
1DL301,0.0,0.0,0.0,0.0
1DL311,0.0,0.0,0.0,0.0
...,...,...,...,...
inf-EAeS,9.0,30.0,39.0,168.0
inf-EERP,15.0,51.0,39.0,168.0
inf-EnlOpt,3.0,24.0,39.0,168.0


In [39]:
len(stage_two_enrolment.reset_index()['elective'].unique())

294